### Objective: Manipulate (spatio-temporal) data through SQL

- Citibike ridership data: https://serv.cusp.nyu.edu/~hvo/files/citibike.csv
- Interactive SQL queries using Carto: https://carto.com

In [1]:
# function by @fedhere to import query results

SQL_SOURCE = 'https://xoxoaseka.carto.com/api/v2/sql?q='

import urllib2
import urllib
import StringIO
import ast
import pandas as pd

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    '''queries carto datasets from a given carto account
    Arguments: 
    query - string: a valid sql query string
    format - outlut format  OPTIONAL (default CSV)
    source - a valid sql api endpoint OPTIONAL (default carto fb55 account)
    Returns:
    the return of the sql query AS A STRING
    NOTES:
    designed for the carto API, tested only with CSV return format'''
    
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return pd.read_csv(StringIO.StringIO(response.read()), sep=',')

SyntaxError: invalid syntax (<ipython-input-1-508c71a774f1>, line 25)

In [ ]:
test = '''
SELECT *
FROM citibike
'''

In [ ]:
#pd.read_csv(StringIO.StringIO(queryCartoDB(test)), sep=',')
queryCartoDB(test)

### Task 1 — Familiarize with SQL Clauses

1) Sort data by start_station_id, tripduration (only checking trips with duration <= 3 hours)

2) Only show the top/last 10 records (aka head and tail in SQL) List all unique start_station_id values

3) Aggregation functions:
- Count the number of trips (aka wc -l in SQL) 
- Find the average/min/max trip duration

In [ ]:
task1 = '''
SELECT DISTINCT start_station_id, 
       COUNT(start_station_id) as trip_counts,
       AVG(tripduration) as average_trip_duration,
       MIN(tripduration) as min_trip_duration,
       MAX(tripduration) as max_trip_duration
FROM citibike
WHERE tripduration <= 10800
GROUP BY start_station_id
ORDER BY start_station_id ASC
LIMIT 10
'''
#show top 10 records
queryCartoDB(task1)

In [ ]:
task1_1 = '''
SELECT DISTINCT start_station_id, 
       COUNT(start_station_id) as trip_counts,
       AVG(tripduration) as average_trip_duration,
       MIN(tripduration) as min_trip_duration,
       MAX(tripduration) as max_trip_duration
FROM citibike
WHERE tripduration <= 10800
GROUP BY start_station_id
ORDER BY start_station_id DESC
LIMIT 10
'''
# showing last 10 records
queryCartoDB(task1_1)

### Task 2 — Working with date/time

1) Selecting trips started on Feb-02-2015 only 

2) Selecting trips started on the weekends (What are average trip duration during weekends?) 

3) Can we do the same for weekday?

In [ ]:
task2_1 = '''
SELECT *
FROM citibike
WHERE starttime >= '2015-02-02 00:00'
AND starttime < '2015-02-03 00:00'
'''
# showing trips started on Feb-02-2015 only
queryCartoDB(task2_1)

In [ ]:
# showing trips started on the weekends
task2_2_1 = '''
SELECT *
FROM citibike
WHERE extract(DOW from starttime) IN (0,6)
'''
queryCartoDB(task2_2_1)

In [ ]:
# showing average trip duration during weekends
task2_2_2 = '''
SELECT AVG(tripduration)
FROM citibike
WHERE extract(DOW from starttime) IN (0,6) 
'''
queryCartoDB(task2_2_2)

In [ ]:
# showing trips started on the weekdays
task2_3_1 = '''
SELECT *
FROM citibike
WHERE extract(DOW from starttime) IN (1, 2, 3, 4, 5)
'''
queryCartoDB(task2_3_1)

In [ ]:
# showing average trip duration during weekdays
task2_3_2 = '''
SELECT AVG(tripduration)
FROM citibike
WHERE extract(DOW from starttime) IN (1, 2, 3, 4, 5)
'''
queryCartoDB(task2_3_2)

### Task 3 — Working with Space
1) Showing the list of start station locations (using GROUP BY)

2) Showing the number of trips started per station (only for stations within 500m of Time Square! The coordinates of Time Square is (40.7577,-73.9857))

In [ ]:
task3_1 = '''
SELECT start_station_latitude, start_station_longitude 
FROM citibike
GROUP BY start_station_latitude, start_station_longitude
'''
# showing start station locations 
queryCartoDB(task3_1)

In [ ]:
task3_2 = '''
SELECT start_station_name, 
       CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id,
       COUNT(tripduration) as number_of_trips
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography, 500)
GROUP BY start_station_latitude, start_station_longitude, start_station_name
ORDER BY number_of_trips
'''
# showing the number of trips started per station located within 500m of Time Square
queryCartoDB(task3_2)

### Task 4 — Putting it all together
1) Find the station that had the longest average trip duration during weekends and within 500m of TimeSquare!

2) Extra: create lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours. The number of trips per each pair of stations are output as attributes of these lines.

In [ ]:
task4_1 = '''
SELECT start_station_name,
       CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id,
       AVG(tripduration) as average_trip_duration
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography, 500)
      AND extract(DOW from starttime) IN (0,6)
GROUP BY start_station_latitude, start_station_longitude, start_station_name
ORDER BY average_trip_duration DESC
'''
# showing top 10 stations that had the longest average trip duration during weekends and within 500m of TimeSquare
queryCartoDB(task4_1)

In [ ]:
task4_2 = '''
SELECT start_station_name, end_station_name,
       ST_Makeline(CDB_TransformToWebmercator(CDB_LatLng(start_station_latitude, start_station_longitude)),
                   CDB_TransformToWebmercator(CDB_LatLng(end_station_latitude, end_station_longitude))) as the_geom_webmercator,
       MIN(cartodb_id) as cartodb_id,
       COUNT(start_station_id) as trip_counts
FROM citibike
WHERE ST_DWithin(CDB_LatLng(start_station_latitude, start_station_longitude)::geography, 
                 CDB_LatLng(40.7577,-73.9857)::geography, 500)
      AND tripduration < 7200
GROUP BY start_station_latitude, start_station_longitude, start_station_name, end_station_latitude, end_station_longitude, end_station_name
ORDER BY trip_counts DESC
'''
# showing lines for trips started from stations within 500m of Times Squares and lasted less than 2 hours. 
# the number of trips per each pair of stations are output as attributes of these lines.
queryCartoDB(task4_2)

## ASH - Score 100. Extra Credit 0